# Multi-label classification -- top-push loss

In [1]:
%matplotlib inline
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

import os, sys, time
import pickle as pkl
import numpy as np
import pandas as pd

from scipy.optimize import minimize
from scipy.optimize import check_grad

from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sys.path.append('src')
from evaluate import avgPrecision, avgPrecisionK, printEvaluation
from datasets import create_dataset_yeast_train, create_dataset_yeast_test, yeast_nLabels
from datasets import create_dataset_scene_train, create_dataset_scene_test, scene_nLabels
from datasets import create_dataset_mediamill_subset_train, create_dataset_mediamill_subset_test, mm_nLabels

In [3]:
datasets = ['yeast', 'scene', 'mediamill']
num_labels = [yeast_nLabels, scene_nLabels, mm_nLabels]
create_dataset_train_funcs = [create_dataset_yeast_train, 
                              create_dataset_scene_train, 
                              create_dataset_mediamill_subset_train]
create_dataset_test_funcs  = [create_dataset_yeast_test,
                              create_dataset_scene_test,
                              create_dataset_mediamill_subset_test]

In [4]:
data_ix = 0

In [5]:
dataset_name = datasets[data_ix]
nLabels = num_labels[data_ix]
create_dataset_train = create_dataset_train_funcs[data_ix]
create_dataset_test  = create_dataset_test_funcs [data_ix]

The sigmoid function.

In [6]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

## top-push loss

Multi-label learning with top push loss.

In [7]:
def obj_top_push(w, X, Y, C):
    """
        Objective with L2 regularisation and top push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
                 if we normalise the objective J by dividing C, \lambda = 1/C
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == N * K)
    assert(C > 0)
    
    T = w.reshape(N, K)  # theta
        
    J = 0.0  # cost
    G = np.zeros_like(T)  # gradient matrix
    nPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    nNegAll = K - nPosAll        # number of negative labels for each example, N by 1
    
    for k in range(K):
        y = Y[:, k]
        
        posVec = np.zeros(N, dtype=np.float)
        negVec = np.zeros(N, dtype=np.float)
        posVec[y == 1] = 1
        negVec[y == 0] = 1
        posVec = np.divide(posVec, nPosAll * N)  # 1 / NK+, N by 1
        negVec = np.divide(negVec, nNegAll * N)  # 1 / NK-, N by 1
        
        a = np.multiply(T[:, k], posVec)  # alpha / NK+ if y_nk = 1 else 0
        b = np.multiply(T[:, k], negVec)  # beta  / NK- if y_nk = 0 else 0
        c = a + b  # gamma
        d = np.sum(X * c[:, None], axis=0)
        
        t1 = T[y == 1, k]
        p1 = posVec[y == 1]
        s1 = -np.log(-t1)
        s2 = np.log(1 + t1)
        J += 0.5 * C * np.dot(d, d) + np.dot(a[y == 1], s1) + np.dot(p1, np.multiply(1+t1, s2))
        
        p2 = negVec[y == 0]
        t2 = np.dot(X, d) * C
        G[y == 1, k] = np.multiply(p1, t2[y == 1] + s1 + s2)
        G[y == 0, k] = np.multiply(p2, t2[y == 0])
        
    return (J, G.ravel())

In [8]:
def obj_top_push_loop(w, X, Y, C):
    """
        Objective with L2 regularisation and top push loss
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
                 if we normalise the objective J by dividing C, \lambda = 1/C
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == N * K)
    assert(C > 0)
    
    T = w.reshape(N, K)  # theta
    
    J = 0.0  # cost
    G = np.zeros_like(T)  # gradient matrix
    KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    KNegAll = K - KPosAll        # number of negative labels for each example, N by 1
    
    for k in range(K):
        y = Y[:, k]
        
        # cost
        X1 = np.zeros_like(X)
        for n in range(N):
            t = T[n, k] / (N * KPosAll[n]) if y[n] == 1 else T[n, k] / (N * KNegAll[n])
            X1[n, :] = X[n, :] * t
        xs = np.sum(X1, axis=0)
        J += np.dot(xs, xs) * 0.5 * C
        #print(J)
        
        for n in range(N):
            if y[n] == 1:
                #print(T[n,k], n, k)
                assert(T[n, k] < 0)
                J += (-T[n, k] * np.log(-T[n, k]) + (1 + T[n, k]) * np.log(1 + T[n, k])) / (N * KPosAll[n])
        
        # gradient
        for n in range(N):
            if y[n] == 1:
                ga = C * np.dot(np.sum(X1, axis=0), X[n, :]) - np.log(-T[n, k]) + np.log(1 + T[n, k])
                G[n, k] = G[n, k] + ga / (N * KPosAll[n])
            else:
                gb = C * np.dot(np.sum(X1, axis=0), X[n, :])
                G[n, k] = G[n, k] + gb / (N * KNegAll[n])
        
    return (J, G.ravel())

In [9]:
def dual2primal(w, X, Y, C):
    """
        Compute primal variable values given dual variable values
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
                 if we normalise the objective J by dividing C, \lambda = 1/C
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == N * K)
    assert(C > 0)
    
    W = np.zeros(K, D)
    T = w.reshape(N, K)  # theta
    
    nPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    nNegAll = L - nPosAll        # number of negative labels for each example, N by 1
    
    for k in range(K):
        y = Y[:, k]
        
        posVec = np.zeros(N, dtype=np.float)
        negVec = np.zeros(N, dtype=np.float)
        posVec[y == 1] = 1
        negVec[y == 0] = 1
        posVec = np.divide(posVec, nPosAll / N)
        negVec = np.divide(negVec, nNegAll / N)
        
        t = np.multiply(T[:, k], posVec) + np.multiply(T[:, k], negVec)  # gamma
        W[k, :] = -C * np.sum(X * t[:, None], axis=0)
        
    return W

In [10]:
def dual2primal_loop(w, X, Y, C):
    """
        Compute primal variable values given dual variable values
        
        Input:
            - w: current weight vector, flattened L x D
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
            - C: regularisation constant, is consistent with scikit-learn C = 1 / (N * \lambda)
                 if we normalise the objective J by dividing C, \lambda = 1/C
    """
    N, D = X.shape
    K = Y.shape[1]
    assert(w.shape[0] == N * K)
    assert(C > 0)
    
    W = np.zeros(K, D)
    T = w.reshape(N, K)  # theta
    
    KPosAll = np.sum(Y, axis=1)  # number of positive labels for each example, N by 1
    KNegAll = L - KPosAll        # number of negative labels for each example, N by 1
    
    for k in range(K):
        y = Y[:, k]
        X1 = X.copy()
        for n in range(N):
            t = T[n, k] / (N * KPosAll[n]) if y[n] == 1 else T[n, k] / (N * KNegAll[n])
            X1[n, :] = X1[n, :] * t
        W[k, :] = -C * np.sum(X1, axis=0)
        
    return W

In [11]:
#aa = np.array([0,1])
#-1 * (2 * aa - 1)

In [12]:
def initVar(X, Y, seed=None):
    """
        Initialise the dual variables
        
        Input:
            - X: feature matrix, N x D
            - Y: label matrix,   N x K
        
        Output:
            - W: matrix with random numbers 
                 W_{n,k} \in (-1,0) if Y_{n,k} = 1, 
                 W_{n,k} \in (0, 1) if Y_{n,k} = 0.
    """
    N, D = X.shape
    K = Y.shape[1]
    
    I = -1 * (2 * Y - 1)
    
    if seed is not None: 
        np.random.seed(seed)
    
    return np.multiply(np.random.rand(N * K).reshape(N, K), I).ravel()

Check gradient

In [13]:
X_train, Y_train = create_dataset_train()
X_test,  Y_test  = create_dataset_test()

In [14]:
X_train.shape

(1500, 103)

In [15]:
# testing func
def obj_func(w):
    # f = xy + x^2
    x = w[0]
    y = w[1]
    return (x*y + x*x, np.asarray([y + 2*x, x]))

eps = 1.49e-08
w0 = np.random.rand(2)
w = np.zeros_like(w0)
for i in range(len(w0)):
    wi1 = w0.copy()
    wi2 = w0.copy()
    wi1[i] = wi1[i] - eps
    wi2[i] = wi2[i] + eps
    J1, _ = obj_func(wi1)
    J2, _ = obj_func(wi2)
    w[i] = (J2 - J1) / (2 * eps)
J, w1 = obj_func(w0)
wdiff = w1 - w
print(np.dot(wdiff, wdiff))

1.36932887022e-18


In [16]:
C = 1
eps = 1.49e-08
w0 = initVar(X_train[:100, :], Y_train[:100, :])
w = np.zeros_like(w0)
for i in range(len(w0)):
    wi1 = w0.copy()
    wi2 = w0.copy()
    wi1[i] = wi1[i] - eps
    wi2[i] = wi2[i] + eps
    J1, _ = obj_top_push_loop(wi1, X_train[:100, :], Y_train[:100, :], C)
    J2, _ = obj_top_push_loop(wi2, X_train[:100, :], Y_train[:100, :], C)
    w[i] = (J2 - J1) / (2 * eps)
    #print(w[i])
J, w1 = obj_top_push_loop(w0, X_train[:100, :], Y_train[:100, :], C)
diff = w1 - w
np.dot(diff, diff)

3.0664722752211736e-15

In [17]:
C = 1
w0 = initVar(X_train[:100, :], Y_train[:100, :])
check_grad(lambda w: obj_top_push_loop(w, X_train[:100, :], Y_train[:100, :], C)[0], 
           lambda w: obj_top_push_loop(w, X_train[:100, :], Y_train[:100, :], C)[1], w0)

6.9814120533485693e-08

In [18]:
C = 1
w0 = initVar(X_train[:100, :], Y_train[:100, :])
check_grad(lambda w: obj_top_push(w, X_train[:100, :], Y_train[:100, :], C)[0], 
           lambda w: obj_top_push(w, X_train[:100, :], Y_train[:100, :], C)[1], w0)
#%lprun -f obj_top_push check_grad(lambda w: obj_top_push(w, X_train, Y_train, C)[0], \
#                                  lambda w: obj_top_push(w, X_train, Y_train, C)[1], w0)

1.0695865106720472e-07

In [19]:
#%%script false
N = X_train.shape[0]
K = Y_train.shape[1]
print('%15s %15s %15s %15s' % ('J_Diff', 'J_loop', 'J_vec', 'G_Diff'))
for e in range(-6, 10):
    C = 10**(e)
    w0 = initVar(X_train, Y_train)
    J,  G  = obj_top_push_loop(w0, X_train, Y_train, C)
    J1, G1 = obj_top_push(w0, X_train, Y_train, C)
    Gdiff = G1 - G
    #print('%-15g %-15g %-15g' % (J1 - J, J, J1))
    print('%15g %15g %15g %15g' % (J1 - J, J, J1, np.dot(Gdiff, Gdiff)))

         J_Diff          J_loop           J_vec          G_Diff
   -2.33147e-15       -0.505574       -0.505574     3.67793e-36
    7.21645e-16       -0.499802       -0.499802     3.52158e-36
   -3.33067e-16       -0.502555       -0.502555     3.45933e-36
   -7.77156e-16       -0.500039       -0.500039     3.56885e-36
    2.22045e-15       -0.497506       -0.497506     3.49379e-36
    4.44089e-16       -0.504511       -0.504511     3.89773e-36
   -5.55112e-16       -0.502103       -0.502103     3.41304e-36
   -3.33067e-16        -0.49891        -0.49891     3.42844e-36
    8.32667e-16       -0.472686       -0.472686     4.67081e-36
   -7.21645e-16       -0.205267       -0.205267     3.89996e-35
    8.88178e-16         2.48421         2.48421     3.37529e-33
   -5.32907e-14         28.7973         28.7973     3.28946e-31
   -3.41061e-13         307.833         307.833      3.4537e-29
   -4.54747e-12         2896.18         2896.18     2.81577e-27
    2.91038e-11         28541.2         

In [ ]:
class MLC_toppush(BaseEstimator):
    """All methods are necessary for a scikit-learn estimator"""
    
    def __init__(self, p=1, C=1):
        """Initialisation"""
        
        assert C > 0
        self.C = C
        self.trained = False
        
    def fit(self, X_train, Y_train):
        """Model fitting by optimising the objective"""
        
        opt_method = 'SLSQP'
        options = {'disp': True}
        if options['disp']: 
            print('\nC: %g' % self.C)
            
        D = X_train.shape[1]
        L = Y_train.shape[1]
        w0 = np.random.rand(L * D)  # initial guess
        opt = minimize(obj_top_push, w0, args=(X_train, Y_train, self.C), \
                       method=opt_method, jac=True, options=options)
        if opt.success is True:
            self.W = dual2primal(opt.x, X_train, Y_train, self.C)
            self.trained = True
        else:
            sys.stderr.write('Optimisation failed')
            self.trained = False
            
            
    def decision_function(self, X_test):
        """Make predictions (score is real number)"""
        
        assert self.trained is True, "Can't make prediction before training"
        D = X_test.shape[1]
        return np.dot(X_test, self.W.T)
        
    
    def predict(self, X_test):
        """Make predictions (score is boolean)"""
        
        preds = self.decision_function(X_test)
        return (preds > 0)
    
    
    def score(self, X, Y):
        """Compute scoring metric"""
        
        allPreds = self.decision_function(X)
        return avgPrecisionK(Y, allPreds)
    
    # inherit from BaseEstimator instead of re-implement
    #
    #def get_params(self, deep = True):
    #def set_params(self, **params):

In [ ]:
parameters = [{'C': [10**(e) for e in range(-6,-1)]}]

clf = GridSearchCV(MLC_toppush(), parameters, cv=5)
clf.fit(X_train, Y_train)

print("\nBest parameters set found on development set:")
print(clf.best_params_)

In [ ]:
for mean, std, params in zip(clf.cv_results_['mean_test_score'], clf.cv_results_['std_test_score'], \
                             clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
preds_train = clf.decision_function(X_train)
preds_test  = clf.decision_function(X_test)

In [ ]:
print('Training set:')
printEvaluation(Y_train, preds_train)
print()
print('Test set:')
printEvaluation(Y_test, preds_test)

## Result analysis

In [ ]:
precisions_train = [avgPrecision(Y_train, preds_train, k) for k in range(1, nLabels+1)]
precisions_test  = [avgPrecision(Y_test,  preds_test,  k) for k in range(1, nLabels+1)]

In [ ]:
precisionK_train = avgPrecisionK(Y_train, preds_train)
precisionK_test  = avgPrecisionK(Y_test,  preds_test)

In [ ]:
plt.figure(figsize=[10,5])
plt.plot(precisions_train, ls='--', c='r', label='Train')
plt.plot(precisions_test,  ls='-',  c='g', label='Test')
plt.plot([precisionK_train for k in range(nLabels)], ls='-', c='r', label='Train, Precision@K')
plt.plot([precisionK_test  for k in range(nLabels)], ls='-', c='g', label='Test, Precision@K')
plt.xticks(np.arange(nLabels), np.arange(1,nLabels+1))
plt.xlabel('k')
plt.ylabel('Precision@k')
plt.legend(loc='best')
plt.title('MLC w. Top-push Loss on ' + dataset_name + ' dataset')
plt.savefig(dataset_name + '_tp.svg')